In [39]:
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt

In [40]:
#ez igazibol 7-es gyak trolololol
train_img = np.load("src_5/train_images.npy")
train_lbl = np.load("src_5/train_labels.npy")
test_img = np.load("src_5/test_images.npy")
test_lbl = np.load("src_5/test_labels.npy")

In [41]:
print(train_img.shape, train_lbl.shape, test_img.shape, test_lbl.shape)
print(len(train_img))

(5000, 64, 64, 3) (5000, 8) (1899, 64, 64, 3) (1899, 8)
5000


In [42]:
def create_layer(num, input_s, output_s, current_input, is_training, scope_name='conv_layer'):
    with tf.variable_scope(scope_name+'-'+str(num)):
        w = tf.get_variable('w', [3, 3, input_s, output_s])
        b = tf.get_variable('b', [output_s])
        conv = tf.nn.conv2d(current_input, w, strides=[1, 1, 1, 1], padding='SAME')
        conv = tf.layers.batch_normalization(conv, training=is_training)
        biased = tf.add(conv, b)
        activated = tf.nn.relu(biased)
        
        return tf.nn.max_pool(activated, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding='SAME')

In [43]:
def output_layer(output_s, current_input, keep_probability):
    with tf.variable_scope('output_layer'):
        current_shape = current_input.get_shape()
        feature_length = int(current_shape[1] * current_shape[2] * current_shape[3])
        fully_connected = tf.reshape(current_input, [-1, feature_length])
        fully_connected = tf.nn.dropout(fully_connected, keep_prob=keep_probability)
        
        w = tf.get_variable('w', [feature_length, output_s])
        b = tf.get_variable('b', [output_s])
        
        return tf.add(tf.matmul(fully_connected, w), b)
        

In [44]:
def get_loss_opt_acc(learning_rate, desired, output):
    with tf.variable_scope('loss'):
        loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(desired, output))
    with tf.variable_scope('optimizer'):
        opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    with tf.variable_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(desired, 1), tf.argmax(output, 1))
        acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    with tf.variable_scope('batch_norm'):
        extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        euo = extra_update_ops
    
    return [loss, acc, opt, euo]

In [45]:
input_size = [train_img.shape[-3], train_img.shape[-2], train_img.shape[-1]]
output_size = train_lbl.shape[-1]

batch_length = 8
learning_rate = 1e-4
num_iteration = 500
num_kernels = [input_size[-1], 32, 64, 128]

tf.reset_default_graph()

inputs = tf.placeholder(tf.float32, [None] + input_size)
desired = tf.placeholder(tf.float32, [None, output_size])
is_training = tf.placeholder(tf.bool)
keep_prob = tf.placeholder(tf.float32)

next_input = inputs
for ind, kernel in enumerate(num_kernels):
    if num_kernels[-1] != kernel:
        next_input = create_layer(ind, kernel, num_kernels[ind+1], next_input, is_training)
    else:
        output = output_layer(output_size, next_input, keep_prob)

acc_plot = np.zeros(num_iteration)
loss_plot = np.zeros(num_iteration)

loss_opt_acc = get_loss_opt_acc(learning_rate, desired, output)
with tf.Session() as ss:
    ss.run(tf.global_variables_initializer())
    
    for i in range(num_iteration):
        used_in_batch = random.sample(range(train_img.shape[0]), batch_length)
        xs_batch = np.reshape(train_img[used_in_batch, :], [batch_length] + input_size)
        ys_batch = train_lbl[used_in_batch, :]
        
        loss, acc, _, _ = ss.run(loss_opt_acc, feed_dict={inputs: xs_batch, desired: ys_batch, is_training: True,
                                                          keep_prob: 0.8})
        acc_plot[i] = acc
        loss_plot[i] = loss
        
        if i%100 == 0:
            print(str(i) + '.: Loss: ' + str(loss) + ' >>> Acc: ' + str(acc))
           
    summ_of_acc = 0.0
    for i in range(200): # test_img
        xs_batch = np.reshape(test_img[i, :], [1] + input_size)
        ys_batch = np.reshape(test_lbl[i, :], [1, output_size])
        
        loss, acc, _, _ = ss.run(loss_opt_acc, feed_dict={inputs: xs_batch, desired: ys_batch, is_training: False,
                                                          keep_prob: 1.0})
        summ_of_acc += acc
    
    print('The avarage accuracy is ', summ_of_acc/200)

0.: Loss: 2.0859451 >>> Acc: 0.125
100.: Loss: 9.184227 >>> Acc: 0.625
200.: Loss: 1.2626019 >>> Acc: 0.875
300.: Loss: 9.181558 >>> Acc: 0.625
400.: Loss: 0.0 >>> Acc: 1.0
The avarage accuracy is  1.0


FailedPreconditionError: Attempting to use uninitialized value output_layer/w
	 [[node output_layer/w/read (defined at <ipython-input-33-c182b72c8fd8>:8)  = Identity[T=DT_FLOAT, _class=["loc:@optimizer/Adam/update_output_layer/w/ApplyAdam"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](output_layer/w)]]

Caused by op 'output_layer/w/read', defined at:
  File "C:\Python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Python36\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Python36\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asyncio_loop.run_foreve